In [2]:
from ase import io
import os

In [7]:
root = "../will_data_061723/original/"
atoms_full = []
for file in os.listdir(root):
    if file.endswith(".xyz"):
        frames = io.read(os.path.join(root, file), index=":")

        for atoms in frames:
            # del atoms.info["magmom"]
            # del atoms.info["dipole"]
            for atom in atoms:
                if "magmom" in atom.data:
                    del atom.data["magmom"]

        atoms_full.extend(frames)


# io.write(os.path.join(root, "will_full_test.xyz"), atoms_full, magmom=False)
def get_header_str(frame, write_stress=False, write_magmom=False):
    if write_stress:
        stress_str = " ".join([str(i) for i in frame.get_stress().flatten()])
    if write_magmom:
        magmom = frame.info["magmom"]

    lattice_str = " ".join([str(i) for i in frame.get_cell().flatten()])
    ret_str = (
        'Lattice="{}" Properties=species:S:1:pos:R:3:forces:R:3 energy={} '.format(
            lattice_str, frame.info["energy"]
        )
    )
    if write_stress:
        ret_str += 'stress="{}" '.format(stress_str)
    if write_magmom:
        ret_str += 'magmom="{}" '.format(magmom)
    ret_str += 'free_energy={} pbc="T T T"\n'.format(frame.info["energy"])
    return ret_str


def custom_writer(filepath, atoms, write_stress=False, write_magmom=False):
    with open(filepath, "w") as f:
        for frame in atoms:
            forces = frame.get_forces()
            n_atoms = len(frame)
            header_str = get_header_str(frame, write_stress, write_magmom)
            f.write("{}\n".format(n_atoms))
            f.write(header_str)
            for ind, i in enumerate(frame):
                f.write(
                    "{:2} {:14.8f} {:16.8f} {:16.8f} {:16.8f} {:16.8f} {:16.8f}\n".format(
                        i.symbol.ljust(0),
                        i.position[0],
                        i.position[1],
                        i.position[2],
                        forces[ind][0],
                        forces[ind][1],
                        forces[ind][2],
                    )
                )


custom_writer(os.path.join(root, "will_full.xyz"), atoms_full)

In [8]:
will_train_atoms = io.read(os.path.join(root, "will_full.xyz"), index=":")

In [9]:
# will file train full
file_will_train = "../will_data_061723/train/will_train_full.xyz"
will_train_atoms = io.read(os.path.join(root, file), index=":")
root_old_data = "../data_prior/"
root_merge_data = "../data_merge_will/"

# iterate over all files in old data
for file in os.listdir(root_old_data):
    if file.endswith(".xyz"):
        atoms = io.read(os.path.join(root_old_data, file), index=":")
        atoms.extend(will_train_atoms)
        file_new = file[:-4] + "_will.xyz"
        custom_writer(
            os.path.join(root_merge_data, file_new),
            atoms,
            write_magmom=False,
            write_stress=False,
        )
        # io.write(os.path.join(root_merge_data, file_new), atoms)

In [10]:
will_train_atoms = io.read("/home/santiagovargas/dev/ml_PES_utils/data_merge_will/target_md_aug_full_test_will.xyz", index=":")

In [12]:
len(will_train_atoms)

1366